In [1]:
from dataclasses import dataclass
import numpy as np
import torch
import math
from botorch.generation import MaxPosteriorSampling
from torch.quasirandom import SobolEngine
import botorch
import gpytorch
import pandas as pd

In this notebook, we will see how to use the TuRBO Bayesian Optimization tool for the capstone project. TuRBO is a BO algorithm proposed by Uber that specializes in high-dimensional problems. You can read the details of the algorithm in the paper:


Eriksson et al., "Scalable Global Optimization via Local Bayesian Optimization", NeurIPS (2019). URL: https://proceedings.neurips.cc/paper/2019/file/6c990b7aca7bc7058f5e98ea909e924b-Paper.pdf

For implementing the method, we will be using the Gaussian Process library GPyTorch, and the Bayesian Optimization library BoTorch. We will be loosely following the tutorial made by BoTorch's team:

https://botorch.org/tutorials/turbo_1

However, we will be making some modification that are case-specific for us.

TuRBO works by creating a Trust Region over which will focus all our optimization efforts. This works great for higher-dimensions because the search space is too large and algorithms tend to over-explore! 

We keep track of a 'Turbo State' that dictates the size and location of the region. The code below implements a data class that will help us keep track of the state:

In [4]:
# we define a dataclass for our state
@dataclass
class TurboState:
    dim: int # dimension of the problem, aka input dimension
    batch_size: int = 1 # we could do batch optimization, but the capstone only does one query at a time
    length: float = 0.8 # the length of the current trust region
    length_min: float = 0.5 ** 7 # minimum length for the trust region
    length_max: float = 1.6 # maximum length for the trust region
    failure_counter: int = 0 # initialize counter of the number of failures to improve on the best observation
    failure_tolerance: int = float("nan")  # Note: Post-initialized
    success_counter: int = 0 # initialize counter of the number of success to improve on the best observation
    success_tolerance: int = 3  # Note: The original paper uses 3, this is the number of successes in a row needed to expand the region
    best_value: float = -float("inf") # best value so far, initialized to be the infimum
    restart_triggered: bool = False 

    def __post_init__(self):
        self.failure_tolerance = math.ceil(
            max([4.0 / self.batch_size, float(self.dim) / self.batch_size]) # number of failures needed in a row to shrink the trust region
        )


def update_state(state, Y_next):
    # count if a success, otherwise a failure
    if max(Y_next) > state.best_value + 1e-3 * math.fabs(state.best_value):
        state.success_counter += 1
        state.failure_counter = 0
    else:
        state.success_counter = 0
        state.failure_counter += 1
    # check if we need to expand or shrink the trust region
    if state.success_counter == state.success_tolerance:  # Expand trust region
        state.length = min(2.0 * state.length, state.length_max)
        state.success_counter = 0
    elif state.failure_counter == state.failure_tolerance:  # Shrink trust region
        state.length /= 2.0
        state.failure_counter = 0
    # set the best value if we got a new observation
    state.best_value = max(state.best_value, max(Y_next).item())
    if state.length < state.length_min:
        state.restart_triggered = True
    return state

It will be very important to keep track of the state when we optimize, as we will need to make sure we keep the state updated from query to query. You can use a print statement to see the value of a state:

In [6]:
state = TurboState(dim = 6)
print(state)

TurboState(dim=6, batch_size=1, length=0.8, length_min=0.0078125, length_max=1.6, failure_counter=0, failure_tolerance=6, success_counter=0, success_tolerance=3, best_value=-inf, restart_triggered=False)


It is important to record these variables after choosing a new query, and re-input, and update to the correct state when we receive new observations. An example of this will be given later. We can then define the TuRBO loop:

In [8]:
def generate_batch(
    state,
    model,  # GP model
    X,  # Evaluated points on the domain [0, 1]^d
    Y,  # Function values
    batch_size = 1, # fix batch size to 1
    n_candidates=None,  # Number of candidates for Thompson sampling
    num_restarts=10,
    raw_samples=512,
    acqf="ts",  # "ei" or "ts"
):
    assert acqf in ("ts")
    assert X.min() >= 0.0 and X.max() <= 1.0 and torch.all(torch.isfinite(Y))
    if n_candidates is None:
        n_candidates = min(5000, max(2000, 200 * X.shape[-1]))

    # Scale the trust region to be proportional to the lengthscales
    x_center = X[Y.argmax(), :].clone()
    weights = model.covar_module.base_kernel.lengthscale.squeeze().detach()
    weights = weights / weights.mean()
    weights = weights / torch.prod(weights.pow(1.0 / len(weights)))
    tr_lb = torch.clamp(x_center - weights * state.length / 2.0, 0.0, 1.0)
    tr_ub = torch.clamp(x_center + weights * state.length / 2.0, 0.0, 1.0)
    # we focus only on thompson sampling as an acquisition function
    if acqf == "ts":
        dim = X.shape[-1]
        sobol = SobolEngine(dim, scramble=True)
        pert = sobol.draw(n_candidates)
        pert = tr_lb + (tr_ub - tr_lb) * pert

        # Create a perturbation mask
        prob_perturb = min(20.0 / dim, 1.0)
        mask = (
            torch.rand(n_candidates, dim)
            <= prob_perturb
        )
        ind = torch.where(mask.sum(dim=1) == 0)[0]
        mask[ind, torch.randint(0, dim - 1, size=(len(ind),))] = 1

        # Create candidate points from the perturbations and the mask        
        X_cand = x_center.expand(n_candidates, dim).clone()
        X_cand[mask] = pert[mask]

        # Sample on the candidate points
        # set model to evaluation mode
        model.eval()
        posterior_distribution = model(X_cand)
        with torch.no_grad():  # We don't need gradients when using TS
            posterior_sample = posterior_distribution.sample()
            X_next_idx = torch.argmax(posterior_sample)
            X_next = X_cand[X_next_idx]

    return X_next

The function above requires us to use a GPyTorch model as an input. A tutorial on how GPyTorch models can be used is found here: https://docs.gpytorch.ai/en/stable/examples/01_Exact_GPs/Simple_GP_Regression.html

Below we define our model class:

In [10]:
# we use the model given in the tutorial, we also add the hyper-parameter training as a method
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        # set a constant mean
        self.mean_module = gpytorch.means.ConstantMean()
        # use a simple RBF kernel with constant scaling
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=train_x.shape[1]))
        # set number of hyper-parameter training iterations
        self.training_iter = 2000

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

For the purposes of this notebook, we will optimize the function:

$$f(x_1, x_2, x_3, x_4, x_5, x_6) = x_3 * \sin(x_1) * \cos(x_2) + x_4 * x_5 - x_6 * x_5^2$$

We will create an initial data set at random.

Do not forget to re-define our state as we have a new best-observation!

We now need to train the hyper-parameters of the model. This can be done in a similar fashion to a normal PyTorch model.

All we need is to define a model and a likelihood, and then activate .train() mode. We then follow classical PyTorch syntax:

We can now define a function that takes as input:
1. Training Data
2. A TuRBO State

And returns the next suggested query! We will define the GP model and optimize the GP's hyper-parameters inside the function itself.

In [14]:
def next_query_via_TurBO(train_x, train_y, turbo_state, verbose = True):
    # the verbose variable decides wether to print the hyper-parameter optimization details
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(train_x, train_y, likelihood)

    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


    #for i in range(model.training_iter):
    #    # Zero gradients from previous iteration
    #    optimizer.zero_grad()
    #    # Output from model
    #    output = model(train_x)
    #    # Calc loss and backprop gradients
    #    loss = -mll(output, train_y)
    #    loss.backward()
    #    if (i % 10 == 9) & verbose:
    #        print(f'Iter %d/%d - Loss: %.3f   lengthscale: {model.covar_module.base_kernel.lengthscale}   noise: %.3f' % (
    #            i + 1, model.training_iter, loss.item(),
    #            model.likelihood.noise.item()
    #        ))
    #    optimizer.step()
    
    return generate_batch(turbo_state, model = model, X = train_x, Y = train_y)

Finally, we can obtain a suggested query!

In [16]:
functions = np.array([i+1 for i in range(8)])
functions

dimensions = np.array([2,2,3,4,4,5,6,8])
dimensions

new_points = pd.read_csv('574_data.csv')
new_points.head()

,Unnamed: 0,timestamp,student_id,f1,f2,f3,f4,f5,f6,f7,f8,f1_output,f2_output,f3_output,f4_output,f5_output,f6_output,f7_output,f8_output
0,3,"03/05/2024,07:48:46",574,"[0.001256,0.001021]","[0.003918,0.999461]","[0.968618,0.009863,0.975288]","[0.005633,0.985666,0.980462,0.996151]","[0.972187,0.993826,0.96857,0.993006]","[0.031377,0.001666,0.983761,0.039847,0.86586,]","[0.091454,0.865694,0.972187,0.993826,0.96857,0...","[0.068548,0.966567,0.010166,0.017496,0.948114,...",6.010000e-247,-0.030595,-0.409314,-45.929543,7364.257265,-2.548909,0.000366,8.975812
1,4,"03/05/2024,07:50:03",574,"[3.84000e-04,9.99836e-01]","[0.999589,0.001177]","[0.980224,0.987284,0.984398]","[0.022037,0.959062,0.973918,0.998504]","[0.997074,0.019557,0.967915,0.016323]","[0.011266,0.034915,0.985681,0.973807,0.946154]","[0.934126,0.948954,0.940902,0.048562,0.97165,0...","[0.017093,0.872336,0.025212,0.878976,0.9981,0....",0.000000e+00,0.118484,-0.417066,-44.550293,1348.169270,-2.115151,0.001524,8.594236
2,5,"03/05/2024,07:51:27",574,"[0.997409,0.002207]","[1.083e-03,7.900e-05]","[0.993846,0.004918,0.978574]","[0.006497,0.001336,0.998308,0.039455]","[0.944327,0.990429,0.003818,0.011438]","[0.96377,0.081122,0.030974,0.009299,0.986679]","[0.080539,0.951776,0.970393,0.126416,0.001409,...","[0.979727,0.034578,0.99476,0.902306,0.697474,0...",0.000000e+00,0.000554,-0.406942,-31.907236,1148.274207,-3.030563,0.070004,5.364045
3,47,"07/05/2024,11:28:08",574,"[7.09823e-01,1.00000e-06]","[0.999999,0.999999]","[4.36557e-01,9.99999e-01,1.00000e-06]","[0.410877,0.386597,0.384497,0.408079]","[0.999999,0.999999,0.999999,0.999999]","[1.00000e-06,1.21733e-01,1.73952e-01,9.99999e-...","[1.00000e-06,1.00000e-06,2.41238e-01,1.00000e-...","[0.182384,0.376578,0.075565,0.305891,0.497701,...",-1.290000e-182,-0.010824,-0.151982,0.318992,8662.405001,-1.468484,0.408992,9.838246
4,48,"07/05/2024,11:37:10",574,"[0.997527,0.346407]","[0.00044,0.216108]","[9.38553e-01,7.79000e-04,9.92212e-01]","[0.099045,0.983822,0.983594,0.876675]","[0.446814,0.422643,0.636614,0.692528]","[5.64120e-02,7.21999e-01,9.38553e-01,7.79000e-...","[0.727272,0.326541,0.570444,0.520834,0.961172,...","[2.45056e-01,5.14880e-01,1.00000e-06,2.98890e-...",-4.020000e-151,0.049944,-0.465217,-39.372590,0.889370,-2.582412,0.007184,9.687430


In [27]:
for i in functions:
    train_x = torch.from_numpy(np.load('initial_data/function_'+str(i)+'/initial_inputs.npy')).to(torch.float32)
    train_y = torch.from_numpy(np.load('initial_data/function_'+str(i)+'/initial_outputs.npy')).to(torch.float32)
    train_x_2 = torch.from_numpy(np.load('initial_data2/function_'+str(i)+'/initial_inputs.npy')).to(torch.float32)
    train_y_2 = torch.from_numpy(np.load('initial_data2/function_'+str(i)+'/initial_outputs.npy')).to(torch.float32)
    
    x_new = []
    for j in range(len(new_points.index)):
        point = np.fromstring(str(new_points['f'+str(i)][j])[1:-1],sep=',')
        x_new = np.concatenate((x_new,point),axis=0)

    x_new = torch.from_numpy(np.reshape(x_new,(-1,dimensions[i-1])))
    x_new
    y_new = torch.from_numpy(new_points['f'+str(i)+'_output'].values)
    y_new
    state = TurboState(dim = dimensions[i-1], best_value = torch.max(train_y).float())
    state
    train_x = torch.concat((train_x, train_x_2, x_new))
    train_x
    train_y = torch.concat((train_y, train_y_2, y_new))
    train_y
    next_query = next_query_via_TurBO(train_x=train_x, train_y=train_y, turbo_state=state)
    print(f'Next chose query for function {i}:',np.array2string(next_query.numpy(), precision=6, separator='-', floatmode='fixed',formatter={'float': '{:0.6f}'.format}))
    #print('Best X: ',np.array2string(np.array(X_turbo[-1]), precision=6, separator='-', floatmode='fixed',formatter={'float': '{:0.6f}'.format}))

C:\Users\T1934127\AppData\Local\anaconda3\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


Next chose query for function 1: [0.630253-0.217094]
Next chose query for function 2: [0.889881-0.837831]
Next chose query for function 3: [0.565764-0.960095-0.856322]
Next chose query for function 4: [0.141256-0.477034-0.282691-0.179128]
Next chose query for function 5: [0.995092-0.977907-0.965886-0.950196]
Next chose query for function 6: [0.006491-0.223172-0.216323-0.343592-0.261476]
Next chose query for function 7: [0.126062-0.419962-0.397820-0.015457-0.021181-0.957438]
Next chose query for function 8: [0.321223-0.459460-0.435945-0.416337-0.419268-0.489644-0.615212-0.425271]
